In [1]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.nn import *
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
X_train = [[0, 1, 1], [1,1,1]]
y_train = [1,0]
X_test = [[0, 1, 1], [1,1,1]]
y_test = [1,0]

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y).unsqueeze(1)
 
    def __len__(self):
        return len(self.X)
 
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [32]:
batchSize = 1
dataset_train = Dataset(X_train, y_train)
dataset_test = Dataset(X_test, y_test)
train_dl = torch.utils.data.DataLoader(dataset_train, batch_size=batchSize, shuffle=False)
test_dl = torch.utils.data.DataLoader(dataset_test, batch_size=batchSize, shuffle=False)

In [119]:
class MLP(torch.nn.Module):
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        emb_size = 8
        self.layer1 = Linear(emb_size+n_inputs, 10)
        self.layer2 = Linear(10, 10)
        self.layer3 = Linear(10, 1)
        self.activation = LeakyReLU()
        self.embedding = torch.rand([3, emb_size], requires_grad=True).to(device)

 
    def forward(self, X):
        codes = X[:, 0].long()
        X = X[:, 1:]
        emb = self.embedding[codes]
        X = torch.cat((X, emb), 1)
        X = self.activation(self.layer1(X))
        X = self.layer3(X)
        return X

In [126]:
model = MLP(2).to(device)

In [127]:
criterion = torch.nn.MSELoss()   
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1, weight_decay=1e-8)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, min_lr=1e-6, factor=0.5)

In [128]:
start_time = datetime.now()

for epoch in range(101):
    train_loss = 0
    model.train()
    for i, (inputs, targets) in enumerate(train_dl):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        yhat = model(inputs)
        loss = criterion(yhat, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss
        
    model.eval()
    
    with torch.no_grad():
        eval_loss = 0
        for i, (inputs, targets) in enumerate(test_dl):
            inputs, targets = inputs.to(device), targets.to(device)
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            eval_loss += loss
        
    #scheduler.step(eval_loss)
    
    if epoch % 100 == 0:
        print("Epoch: " + str(epoch) + ", train loss: " + str(train_loss/len(train_dl)) + ", eval loss: " + str(eval_loss/len(test_dl)))

print(datetime.now() - start_time)

Epoch: 0, train loss: tensor(0.2777, device='cuda:0', grad_fn=<DivBackward0>), eval loss: tensor(0.0656, device='cuda:0')
Epoch: 100, train loss: tensor(3.5533e-06, device='cuda:0', grad_fn=<DivBackward0>), eval loss: tensor(3.7253e-06, device='cuda:0')
0:00:00.298257


In [129]:
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_dl):
        print(inputs, targets)
        inputs, targets = inputs.to(device), targets.to(device)
        yhat = model(inputs)
        print(yhat)
        #break

tensor([[0., 1., 1.],
        [1., 1., 1.]]) tensor([[1.],
        [0.]])
tensor([[ 0.9975],
        [-0.0011]], device='cuda:0')
